# Obteniendo el TOTAL de personas totalmente vacunadas (2 dosis) y de muertes confirmadas por COVID-19 de cada uno de los 24 departamentos del Perú

En este notebook se busca obtener un dataset con el total de **fallecidos** y **totalmente vacunados** **(2 dosis)** por cada **departamento** de todo el Perú. De la misma manera se agrega el **número de habitantes** de cada departamento, para calcular la **tasa de mortalidad por 100 000 habitantes** y el **porcentaje de vacunación** respectivo.

Se considera como **completamente vacunado** a todas las personas que recibieron 2 dosis de vacunación.

## 0. Cargar librerías

In [1]:
import pandas as pd
import dask.dataframe as dd

## 1. Procesar del dataset de fallecidos

### 1.1. Leer y procesar el dataset

In [2]:
url_fal = 'RawData/fallecidos_covid.csv'
fal_col = ['DEPARTAMENTO']

df_fal =  pd.read_csv(url_fal, sep = ';', usecols = fal_col, dtype = {'DEPARTAMENTO': 'category'})
df_fal['fallecidos'] = 1    # Contador para cada fallecido
del url_fal, fal_col

df_fal.head()

,DEPARTAMENTO,fallecidos
0,PIURA,1
1,PIURA,1
2,LIMA,1
3,JUNIN,1
4,LAMBAYEQUE,1


Se cuenta a cada caso con el contador creado anteriormente respecto a los 24 departamentos.

In [3]:
df_fal = df_fal.groupby('DEPARTAMENTO').count()
df_fal

,fallecidos
DEPARTAMENTO,
AMAZONAS,1289
ANCASH,6773
APURIMAC,1528
AREQUIPA,9770
AYACUCHO,2164
CAJAMARCA,4194
CALLAO,10123
CUSCO,4836
HUANCAVELICA,1185


Ahora tenemos el TOTAL de fallecidos por cada departamento del Perú.

## 2. Procesar el dataset de vacunados

No existe una manera directa de calcular el total de personas completamente vacunadas por departamento. Para lograrlo se prevee lo siguiente:

- El dataset de vacunados (RawData/TB_VACUNACION_COVID19.csv) solo proporciona información sobre el centro de vacunación llamado **'id_centro_vacunacion'**. NO el departamento.

- El dataset de los centros de vacunación [(RawData/TB_CENTRO_VACUNACION.csv)]() se puede utilizar para hacer **"match"** del **'id_centro_vacunacion'** del dataset de vacunados con la variable **'id_ubigeo'**. Que es una variable numérica de 0 a 1894 que representa a cada distrito.

- Finalmente, con el dataset de UBIGEO [(RawData/TB_UBIGEOS.csv)]() es posible reemplazar a cada **'id_ubigeo'** con el departamento respectivo.

### 2.1. Leer el dataset


In [4]:
url_vac = 'RawData/TB_VACUNACION_COVID19.csv'
vac_col = ['id_eess', 'dosis']

df_vac = dd.read_csv(url_vac, sep = ",", usecols = vac_col, dtype = {'id_eess':'int32','dosis': 'category'})
del url_vac, vac_col

In [5]:
df_vac = df_vac.categorize()
dosis = dd.get_dummies(df_vac['dosis'])
del dosis['4'], dosis['5']

df_vac = dd.concat([df_vac, dosis], axis=1, ignore_unknown_divisions=True)
del dosis, df_vac['dosis']

df_vac.head()

,id_eess,1,2,3
0,14518,1,0,0
1,20298,1,0,0
2,15366,0,1,0
3,3320,1,0,0
4,770,0,1,0


### 2.2. Procesar los datasets de **UBIGEO** y **CENTROS DE VACUNACIÓN**

#### 2.2.1. Procesar los datasets

Para el dataset de UBIGEO solo se necesita las columnas 'id_ubigeo' y 'departamento'. Para el dataset de CENTROS DE VACUNACIÓN solo se necesitan las columnas 'id_centro de vacunación' e 'id_ubigeo'.

In [6]:
# Leemos los datasets de ubigeo y centros de vacunación
url_ubigeo = 'RawData/TB_UBIGEOS.csv'
url_vaccenter = 'RawData/TB_EESS.csv'

ubigeo = pd.read_csv(url_ubigeo, usecols = ['id_ubigeo', 'departamento'])
vaccenter = pd.read_csv(url_vaccenter, usecols= ['id_eess','id_ubigeo'])

In [7]:
# Unimos ambos dataset mediante 'id_ubigeo'
vaccenter = vaccenter.merge(ubigeo, on = 'id_ubigeo', how = 'left')
del ubigeo, vaccenter['id_ubigeo'], url_vaccenter, url_ubigeo

vaccenter.head()

,id_eess,departamento
0,5845,LORETO
1,15599,LORETO
2,18282,LORETO
3,18156,LORETO
4,858,LORETO


Ahora con el nuevo dataset **(vaccenter)** es posible encontrar el departamento correspondiente a 'id_centro de vacunacion' del dataset de vacunados **(df_vac)**.

#### 2.2.2. Encontrar el departamento correspondiente de todas las personas vacunadas

In [8]:
df_vac = df_vac.merge(vaccenter, on = 'id_eess', how = 'left')
del df_vac['id_eess'], vaccenter

df_vac.head()

,1,2,3,departamento
0,1,0,0,APURIMAC
1,1,0,0,CAJAMARCA
2,0,1,0,LA LIBERTAD
3,1,0,0,LIMA
4,0,1,0,LIMA


### 2.3. Agrupar a todos los vacunados por departamento

#### 2.3.1. Contar el total de número de dosis por departamento

In [9]:
df_vac = df_vac.groupby('departamento').agg({'1': ['sum'], '2': ['sum'], '3': ['sum']}).compute()
df_vac = df_vac.astype({('1', 'sum'): 'int64', ('2', 'sum'): 'int64', ('3', 'sum'): 'int64'})

df_vac

,1,2,3
,sum,sum,sum
departamento,,,
AMAZONAS,262837,223698,36559
ANCASH,918972,851867,177289
APURIMAC,343343,296213,43277
AREQUIPA,1164336,1054705,201578
AYACUCHO,429579,338066,34070
CAJAMARCA,1034938,879126,96149
CALLAO,1047675,969070,226774
CUSCO,984789,848645,110218


## 3. Unir el dataset del TOTAL de muertes confirmadas y TOTAL vacunados por departamento

In [10]:
vacfal_dep = pd.concat([df_fal, df_vac], axis=1)

vacfal_dep.reset_index(level=0, inplace=True)
vacfal_dep.rename(columns = {'index':'departamentos', ('1', 'sum'):'dosis_1', 
                             ('2', 'sum'): 'dosis_2', ('3', 'sum'):'dosis_3'}, inplace = True)

del df_fal, df_vac
vacfal_dep

,departamentos,fallecidos,dosis_1,dosis_2,dosis_3
0,AMAZONAS,1289,262837,223698,36559
1,ANCASH,6773,918972,851867,177289
2,APURIMAC,1528,343343,296213,43277
3,AREQUIPA,9770,1164336,1054705,201578
4,AYACUCHO,2164,429579,338066,34070
5,CAJAMARCA,4194,1034938,879126,96149
6,CALLAO,10123,1047675,969070,226774
7,CUSCO,4836,984789,848645,110218
8,HUANCAVELICA,1185,245100,191277,21004
9,HUANUCO,2725,521499,423733,65443


## 4. Procesar el dataset por el número de habitantes correspondiente a cada departamento

Datos de población estimada para el 2020 obtenida del reporte [«PERÚ:Estimaciones y Proyecciones de Población por Departamento, Provincia y Distrito, 2018 - 2020»](https://www.inei.gob.pe/media/MenuRecursivo/publicaciones_digitales/Est/Lib1715/Libro.pdf). Página 39 estimaciones de población total proyectada para 2020. Fuente: [INEI](https://www.inei.gob.pe/).

### 6.1. Agregar un diccionario con el número de habitantes de cada departamento

In [11]:
dic_dep = {
    "AMAZONAS"	:	426806,
    "ANCASH"	:	1180638,
    "APURIMAC"	:	430736,
    "AREQUIPA"	:	1497438,
    "AYACUCHO"	:	668213,
    "CAJAMARCA"	:	1453711,
    "CALLAO"	:	1129854,
    "CUSCO"	    :	1357075,
    "HUANCAVELICA":	365317,
    "HUANUCO"	:	760267,
    "ICA"	    :	975182,
    "JUNIN"	    :	1361467,
    "LA LIBERTAD":	2016771,
    "LAMBAYEQUE":	1310785,
    "LIMA"	    :	10628470,
    "LORETO"	:	1027559,
    "MADRE DE DIOS":173811,
    "MOQUEGUA"	:	192740,
    "PASCO"	    :	271904,
    "PIURA"	    :	2047954,
    "PUNO"	    :	1237997,
    "SAN MARTIN":	899648,
    "TACNA"	    :	370974,
    "TUMBES"	:	251521,
    "UCAYALI"	:	589110,
    "PERU"  	:	32625948,
}

### 6.2. Procesar el dataset

In [12]:
vacfal_dep['no_habitantes'] = vacfal_dep['departamentos'].map(dic_dep)

# Mortality rate per 100k per department
vacfal_dep['tasa_mortalidad'] = round((vacfal_dep['fallecidos']/vacfal_dep['no_habitantes'])*100000,2)

# % of people fully vaccinated per department
vacfal_dep['vac_porcentaje'] = round((vacfal_dep['dosis_2']*100)/vacfal_dep['no_habitantes'],2)

In [13]:
print(f'Total habitantes: {vacfal_dep.no_habitantes.sum()}')
print(f'Total fallecidos: {vacfal_dep.fallecidos.sum()}')
print(f'Total vacunados con 1 dosis: {vacfal_dep.dosis_1.sum()}')
print(f'Total vacunados con 2 dosis: {vacfal_dep.dosis_2.sum()}')
print(f'Total vacunados con 3 dosis: {vacfal_dep.dosis_3.sum()}')
print(f'Porcentaje de la población vacunada: {round(vacfal_dep.dosis_2.sum()*100/vacfal_dep.no_habitantes.sum(),2)}')

Total habitantes: 32625948
Total fallecidos: 202867
Total vacunados con 1 dosis: 24511772
Total vacunados con 2 dosis: 21611587
Total vacunados con 3 dosis: 4223869
Porcentaje de la población vacunada: 66.24


In [14]:
vacfal_dep

,departamentos,fallecidos,dosis_1,dosis_2,dosis_3,no_habitantes,tasa_mortalidad,vac_porcentaje
0,AMAZONAS,1289,262837,223698,36559,426806,302.01,52.41
1,ANCASH,6773,918972,851867,177289,1180638,573.67,72.15
2,APURIMAC,1528,343343,296213,43277,430736,354.74,68.77
3,AREQUIPA,9770,1164336,1054705,201578,1497438,652.45,70.43
4,AYACUCHO,2164,429579,338066,34070,668213,323.85,50.59
5,CAJAMARCA,4194,1034938,879126,96149,1453711,288.50,60.47
6,CALLAO,10123,1047675,969070,226774,1129854,895.96,85.77
7,CUSCO,4836,984789,848645,110218,1357075,356.35,62.53
8,HUANCAVELICA,1185,245100,191277,21004,365317,324.38,52.36
9,HUANUCO,2725,521499,423733,65443,760267,358.43,55.73


## 7. Guardar el dataset

In [15]:
vacfal_dep.to_csv('Data/DP2_TOTAL_vacunados_y_fallecidos_x_departamento.csv',index = False)

In [16]:
del vacfal_dep